---
# Cairo University Faculty of Engineering
## Machine Learning
## Assignment 3 - Decision trees from scratch

---

**Name** : Ibrahim Mohamed Ibrahim

### Data Set

In [2]:
import numpy as np
import pandas as pd
import time

from collections import Counter
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [3]:
## load data
cardio_df = pd.read_csv('./data/cardio_train.csv' , sep= ';', index_col=0)
cardio_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          70000 non-null  int64  
 1   gender       70000 non-null  int64  
 2   height       70000 non-null  int64  
 3   weight       70000 non-null  float64
 4   ap_hi        70000 non-null  int64  
 5   ap_lo        70000 non-null  int64  
 6   cholesterol  70000 non-null  int64  
 7   gluc         70000 non-null  int64  
 8   smoke        70000 non-null  int64  
 9   alco         70000 non-null  int64  
 10  active       70000 non-null  int64  
 11  cardio       70000 non-null  int64  
dtypes: float64(1), int64(11)
memory usage: 6.9 MB


In [4]:
## view the first 5 element of the data
cardio_df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


### Data Exploration

In [5]:
## convert age to years and round it
cardio_df['age'] = round(cardio_df['age'] / 365.25)
cardio_df.head(5)

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,50.0,2,168,62.0,110,80,1,1,0,0,1,0
1,55.0,1,156,85.0,140,90,3,1,0,0,1,1
2,52.0,1,165,64.0,130,70,3,1,0,0,0,1
3,48.0,2,169,82.0,150,100,1,1,0,0,1,1
4,48.0,1,156,56.0,100,60,1,1,0,0,0,0


In [6]:
## Get dataset statistics
cardio_df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,70000.0,53.303157,6.760171,30.0,48.0,54.0,58.0,65.0
gender,70000.0,1.349571,0.476838,1.0,1.0,1.0,2.0,2.0
height,70000.0,164.359229,8.210126,55.0,159.0,165.0,170.0,250.0
weight,70000.0,74.205690,14.395757,10.0,65.0,72.0,82.0,200.0
ap_hi,70000.0,128.817286,154.011419,-150.0,120.0,120.0,140.0,16020.0
ap_lo,70000.0,96.630414,188.472530,-70.0,80.0,80.0,90.0,11000.0
cholesterol,70000.0,1.366871,0.680250,1.0,1.0,1.0,2.0,3.0
gluc,70000.0,1.226457,0.572270,1.0,1.0,1.0,1.0,3.0
smoke,70000.0,0.088129,0.283484,0.0,0.0,0.0,0.0,1.0
alco,70000.0,0.053771,0.225568,0.0,0.0,0.0,0.0,1.0


### My Decision Tree Class

In [7]:
class MyDecisionTree:

    ## Class Constructor
    def __init__(self, max_depth=5, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.tree = None


    ## Function to calculate the entropy
    def entropy(self, y):

        ## Counter for counting the number of occurrences of each class
        counter = Counter(y)

        ## Frequency of each class
        frequencies  = [count / len(y) for count in counter.values()]

        ## Calculate using entropy formula
        entropy = -sum(frequencies * np.log2(frequencies))

        return entropy

    ## Function to calculate the information gain
    def information_gain(self, X, y, split_attribute):
        split_col = X[:, split_attribute]
        total_entropy = self.entropy(y)

        ## Split the data and target variable based on the split attribute
        left_indices = split_col <= np.median(split_col)
        right_indices = split_col > np.median(split_col)
        y_left, y_right = y[left_indices], y[right_indices]

        ## Calculate the entropy of the left and right groups
        left_entropy, right_entropy = self.entropy(y_left), self.entropy(y_right)

        ## Calculate the information gain
        info_gain = total_entropy - (len(y_left) / len(y)) * left_entropy - (len(y_right) / len(y)) * right_entropy

        return info_gain

    ## Function to get the best split
    def best_split(self, X, y):

        ## variables to save the best information gain and the best attribute index
        best_info_gain = 0
        best_attribute = None

        ## Find the best split attribute
        for i in range(X.shape[1]):
            calculated_info_gain = self.information_gain(X, y, i)
            if calculated_info_gain > best_info_gain:
                best_info_gain = calculated_info_gain
                best_attribute = i

        return best_attribute

    ## Function to build the tree recursively
    def build_tree(self, X, y, depth=0):

        ## Stop recursion if there are no samples left or max depth reached
        if len(y) == 0:
            return Counter(y).most_common(1)[0][0]
        elif depth == self.max_depth:
            return Counter(y).most_common(1)[0][0]

        ## Check if the node is unique
        if len(np.unique(y)) == 1:
            return y[0]

        ## Check if the number of samples is smaller than min_samples_split
        if len(y) < self.min_samples_split:
            return Counter(y).most_common(1)[0][0]

        ## Find the best split attribute and value
        best_attribute = self.best_split(X, y)
        split_value = np.median(X[:, best_attribute])

        ## Split the data based on the best attribute and value
        left_indices = X[:, best_attribute] <= split_value
        right_indices = X[:, best_attribute] > split_value
        X_left, y_left = X[left_indices], y[left_indices]
        X_right, y_right = X[right_indices], y[right_indices]

        ## Check if the number of samples in the split is smaller than min_samples_leaf
        if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
            return Counter(y).most_common(1)[0][0]

        ## Recursively build the left and right branches of the tree
        left_tree = self.build_tree(X_left, y_left, depth+1)
        right_tree = self.build_tree(X_right, y_right, depth+1)

        ## Return the decision node
        return {'attribute': best_attribute, 'split_value':split_value, 'left': left_tree, 'right':right_tree}

    ## Function to get the best parameters
    def fit(self, X, y):
        self.tree = self.build_tree(X, y)

    ## Function to get the prediction
    def predict(self, X):
        y_prediction = []
        for i in range(X.shape[0]):
            node = self.tree
            while isinstance(node, dict):
                if X[i, node['attribute']] <= node['split_value']:
                    node = node['left']
                else:
                    node = node['right']
            y_prediction.append(node)
        return np.array(y_prediction)


In [8]:
cont_columns = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']

for col in cont_columns:
    threshold = cardio_df[col].mean()

    cardio_df['updated_{}'.format(col)] = cardio_df[col].apply(lambda x: 0 if x < threshold else 1)
    cardio_df.drop(col, axis=1, inplace=True)

In [9]:
cardio_df

,gender,cholesterol,gluc,smoke,alco,active,cardio,updated_age,updated_height,updated_weight,updated_ap_hi,updated_ap_lo
id,,,,,,,,,,,,
0,2,1,1,0,0,1,0,0,1,0,0,0
1,1,3,1,0,0,1,1,1,0,1,1,0
2,1,3,1,0,0,0,1,0,1,0,1,0
3,2,1,1,0,0,1,1,0,1,1,1,1
4,1,1,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
99993,2,1,1,1,0,1,0,0,1,1,0,0
99995,1,2,2,0,0,1,1,1,0,1,1,0
99996,2,3,1,0,1,0,1,0,1,1,1,0


In [10]:
## Split the dataset into features and target variable
X = cardio_df.drop(['cardio'], axis=1).values
y = cardio_df['cardio'].values



## Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
##  Create an instance of Sklearn Decision Tree with max_depth 5
skl_tree = DecisionTreeClassifier(criterion= 'entropy', max_depth= 5)

## Fitting decision tree model to data & calculating the time it takes
skl_train_time_start = time.time()
skl_tree.fit(X_train,y_train)
skl_train_time_end = time.time()

## Getting prediction from sklearn tree and calculate the time it takes
skl_pred_time_start = time.time()
skl_tree_y_pred = skl_tree.predict(X_test)
skl_pred_time_end = time.time()

# Calculate accuracy score of Sklearn tree predictions
skl_acc = accuracy_score(skl_tree_y_pred,y_test) * 100

In [12]:
# Create an instance of MyDecisionTree class with max_depth 5
my_tree = MyDecisionTree(max_depth=5)

## Fitting decision tree model to data & calculating the time it takes
my_train_time_start = time.time()
my_tree.fit(X_train, y_train)
my_train_time_end = time.time()


## Getting prediction from  My tree and calculate the time it takes
my_pred_time_start = time.time()
my_tree_y_pred = my_tree.predict(X_test)
my_pred_time_end = time.time()

## Calculate accuracy score of my tree predictions
my_acc = accuracy_score(y_test, my_tree_y_pred) * 100

In [13]:
## Calculate Sklearn tree time
skl_train_time = skl_train_time_end - skl_train_time_start
skl_pred_time = skl_pred_time_end - skl_pred_time_start

## Calculate my tree time
my_train_time = my_train_time_end - my_train_time_start
my_pred_time = my_pred_time_end  - my_pred_time_start

In [14]:
print(f"My Tree training time: {my_train_time}")
print(f"Scikit-Learn training time: {skl_train_time}")
print(50*'-')
print(f"My Tree prediction time: {my_pred_time}")
print(f"Scikit-Learn prediction time: {skl_pred_time}")
print(50*'-')
print(f"My accuracy: {my_acc}")
print(f"Scikit-Learn accuracy: {skl_acc}")

My Tree training time: 1.3094983100891113
Scikit-Learn training time: 0.029889583587646484
--------------------------------------------------
My Tree prediction time: 0.15857625007629395
Scikit-Learn prediction time: 0.0009984970092773438
--------------------------------------------------
My accuracy: 72.97142857142858
Scikit-Learn accuracy: 73.13571428571429
